In [84]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os
import keras

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "autoencoders"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

import time
import tensorlayer as tfl

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


In [85]:
tf.random.set_seed(42)
np.random.seed(42)
batch_size = 32
codings_size = 100 ###################################################

class ResidualBlock(keras.layers.Layer):
    def __init__(self, strides=1):
        super().__init__()
        self.hidden = [keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, padding="SAME", kernel_initializer="he_normal"),
                       keras.layers.BatchNormalization(),
                       keras.layers.PReLU(),
                       keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, padding="SAME", kernel_initializer="he_normal"),
                       keras.layers.BatchNormalization()
                       ]

    def call(self, inputs):
        Z = inputs
        Z1 = inputs
        for layer in self.hidden:
            Z = layer(Z)
            
        return Z+ Z1


In [86]:


class GENERATOR(keras.models.Model):
    def __init__(self):
        super().__init__()
        self.front1 =  keras.models.Sequential([
            keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, kernel_initializer="he_normal", input_shape=[batch_size, codings_size]),###############인풋사이즈 수정?? input_size=[codings_size]
            keras.layers.LeakyReLU(alpha=0.3)
        ])
            
        self.block1 = ResidualBlock()
        self.block2 = ResidualBlock()
        self.block3 = ResidualBlock()
        self.block4 = ResidualBlock()
        self.block5 = ResidualBlock()
        
        self.back1 = keras.models.Sequential([
            keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, kernel_initializer="he_normal"), 
            keras.layers.BatchNormalization()
        ])
        '''
        self.back2 = keras.models.Sequential([
            keras.layers.Conv2D(filters=256, kernel_size=3, strides=1, kernel_initializer="he_normal"),
            tf.nn.depth_to_space(block_size=128)(input),
            keras.layers.PReLU()
        ])
        
        self.back3 = keras.models.Sequential([
            keras.layers.Conv2D(filters=256, kernel_size=3, strides=1, kernel_initializer="he_normal"),
            tf.nn.depth_to_space(block_size=64),
            keras.layers.PReLU()
        ])
        '''
        self.back4 = keras.layers.Conv2D(filters=3, kernel_size=9, strides=1, kernel_initializer="he_normal")

    def call(self, inputs):
        C = self.front1(inputs)
        C1 = self.front1(inputs)
        C=self.block1(C)
        C=self.block2(C)
        C=self.block3(C)
        C=self.block4(C)
        C=self.block5(C)

        C=self.back1(C)
        C2=C+C1
        #C2=self.back2(C2)
        C3=keras.layers.Conv2D(filters=256, kernel_size=3, strides=1, kernel_initializer="he_normal")(C2)
        C4=tf.nn.depth_to_space(block_size=128)(C3),
        C5=keras.layers.PReLU(C4)
        #C2=self.back3(C2)
        C6=keras.layers.Conv2D(filters=256, kernel_size=3, strides=1, kernel_initializer="he_normal")(C5)
        C7=tf.nn.depth_to_space(block_size=128)(C6),
        C8=keras.layers.PReLU(C7)
        
        C9=self.bakc4(C8)
        
        return C9

In [87]:
DISCRIMINATOR = keras.models.Sequential([
    keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, kernel_initializer="he_normal", input_shape=[224,224,3]),
    keras.layers.LeakyReLU(alpha=0.3),
    
    keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.LeakyReLU(alpha=0.3),
    
    keras.layers.Conv2D(filters=128, kernel_size=3, strides=1, kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.LeakyReLU(alpha=0.3),
    
    keras.layers.Conv2D(filters=128, kernel_size=3, strides=2, kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.LeakyReLU(alpha=0.3),
    
    keras.layers.Conv2D(filters=256, kernel_size=3, strides=1, kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.LeakyReLU(alpha=0.3),
    
    keras.layers.Conv2D(filters=256, kernel_size=3, strides=2, kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.LeakyReLU(alpha=0.3),
    
    keras.layers.Conv2D(filters=512, kernel_size=3, strides=1, kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.LeakyReLU(alpha=0.3),
    
    keras.layers.Conv2D(filters=512, kernel_size=3, strides=2, kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.LeakyReLU(alpha=0.3),
    
    keras.layers.Dense(1024,kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.3),
    keras.layers.Dense(1,activation='sigmoid', kernel_initializer="he_normal")
])

generator=GENERATOR()

SRGAN = keras.models.Sequential([generator, DISCRIMINATOR])

ValueError: Input 0 of layer conv2d_223 is incompatible with the layer: : expected min_ndim=4, found ndim=3. Full shape received: [None, 32, 100]

In [79]:
DISCRIMINATOR.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [51]:
def progress_bar(iteration, total, size=30):
    running = iteration < total
    c = ">" if running else "="
    p = (size - 1) * iteration // total
    fmt = "{{:-{}d}}/{{}} [{{}}]".format(len(str(total)))
    params = [iteration, total, "=" * p + c + "." * (size - p - 1)]
    return fmt.format(*params)

In [13]:
def print_status_bar(iteration, total, PSNR, SSIM, Global_loss, size=30):
    end = "" if iteration < total else "\n"
    print("\r{} - PSNR :{} , SSIM :{}, loss :{}".format(progress_bar(iteration, total), PSNR, SSIM, Global_loss), end=end)

In [14]:
import tensorflow_hub as hub
VGG=tf.keras.applications.VGG19(
    include_top=True, weights='imagenet', input_tensor=None, input_shape=None,
    pooling=None, classes=1000, classifier_activation='softmax'
)

VGG.summary()
type(VGG)
def content_loss (inputs, GENERATEING) :
    partial_VGG=VGG[0:11]
    compareSR_VGG=partial_VGG[GENERATEING]
    compareHR_VGG=partial_VGG[inputs]
    loss_var_SR_VGG=np.array(compareSR_HR[10])# shape 56, 56, 256
    loss_var_HR_VGG=np.array(compareSR_HR[10])# shape 56, 56, 256
    loss_pre=(loss_var_SR_VGG-loss_var_HR_VGG)**2
    loss_pre1=loss_pre.sum()
    H,W,CH=loss_var_SR_VGG.shape
    loss_content=loss_pre/(H*W)
    return loss_content


Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [15]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [16]:
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.Nadam(lr=0.01)

NameError: name 'X_train' is not defined

In [ ]:
gan = keras.models.Sequential([generator, discriminator])

In [17]:
def train_dscriminator (X_batch, batch_size, coding_size) :
    noise = tf.random.normal(shape=[batch_size, codings_size])#############################수정
    global generated_image
    generated_images = generator(noise)
    X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
    y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
    discriminator.trainable = True
    with tf.GradientTape() as tape:
        y_pred = discriminator(X_fake_and_real, y1)
        global ad_loss
        ad_loss =- tf.math.log(y_pred)
    gradients = tape.gradient(loss, discriminator.trainable_variables)
    optimizer.apply_gradients(zip(gradients, discriminator.trainable_variables))
    for variable in model.variables:
        if variable.constraint is not None:
            variable.assign(variable.constraint(variable))

In [18]:
def train_generator (X_batch, batch_size, codings_size) :
    noise = tf.random.normal(shape=[batch_size, codings_size])###################################수정
    y2 = tf.constant([[1.]] * batch_size)
    discriminator.trainable = False
    with tf.GradientTape() as tape:
        y_pred=gan(noise, y2)
        global cont_loss
        cont_loss=content_loss(X_batch,generated_images)
    gradients = tape.gradient(loss, gan.trainable_variables)
    optimizer.apply_gradients(zip(gradients, gan.trainable_variables))
    for variable in model.variables:
            if variable.constraint is not None:
                variable.assign(variable.constraint(variable))
    


In [19]:
def train_gan(gan, dataset, batch_size, codings_size, n_epochs=50):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        print("Epoch {}/{}".format(epoch + 1, n_epochs))# not shown in the book
        for step in range(1, n_steps + 1):
            X_batch= dataset[step]##변화 가능
            T_Discriminator(X_batch, batchsize, coding_size)
            T_Generator(X_batch, batch_size, codings_size)
            GLOBAL_loss=cont_loss+ad_loss*10**(-3)
            SSIM=tf.image.ssim(X_batch, generated_images)
            PSNR=tf.image.psnr(X_batch, generated_images)

        print_status_bar(step * batch_size, len(y_train),  PSNR, SSIM, GLOBAL_loss)
    print_status_bar(len(y_train), len(y_train),  PSNR, SSIM, GLOBAL_loss)

